### Define Model

In [109]:
import torchvision

class Decoder(nn.Module):
  def __init__(self, in_channels, middle_channels, out_channels):
    super(Decoder, self).__init__()
    self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
    self.conv_relu = nn.Sequential(
        nn.Conv2d(middle_channels, out_channels, kernel_size=3, padding=1),
        nn.ReLU(inplace=True)
        )
  def forward(self, x1, x2):
    x1 = self.up(x1)
    x1 = torch.cat((x1, x2), dim=1)
    x1 = self.conv_relu(x1)
    return x1

class Unet(nn.Module):
    def __init__(self, n_class):
        super().__init__()

        self.base_model = torchvision.models.resnet18(True)
        self.base_layers = list(self.base_model.children())
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            self.base_layers[1],
            self.base_layers[2])
        self.layer2 = nn.Sequential(*self.base_layers[3:5])
        self.layer3 = self.base_layers[5]
        self.layer4 = self.base_layers[6]
        self.layer5 = self.base_layers[7]
        self.decode4 = Decoder(512, 256+256, 256)
        self.decode3 = Decoder(256, 256+128, 256)
        self.decode2 = Decoder(256, 128+64, 128)
        self.decode1 = Decoder(128, 64+64, 64)
        self.decode0 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(64, 32, kernel_size=3, padding=1, bias=False),
            nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False)
            )
        self.conv_last = nn.Conv2d(64, n_class, 1)

    def forward(self, input):
        e1 = self.layer1(input) # 64,128,128
        e2 = self.layer2(e1) # 64,64,64
        e3 = self.layer3(e2) # 128,32,32
        e4 = self.layer4(e3) # 256,16,16
        f = self.layer5(e4) # 512,8,8
        d4 = self.decode4(f, e4) # 256,16,16
        d3 = self.decode3(d4, e3) # 256,32,32
        d2 = self.decode2(d3, e2) # 128,64,64
        d1 = self.decode1(d2, e1) # 64,128,128
        d0 = self.decode0(d1) # 64,256,256
        out = self.conv_last(d0) # 1,256,256
        return out

In [215]:
class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()
    self.up_1 = nn.Sequential(nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2), nn.ReLU())
    self.up_2 = nn.Sequential(nn.ConvTranspose2d( 64, 32, kernel_size=3, stride=2), nn.ReLU())
    self.up_3 = nn.Sequential(nn.ConvTranspose2d( 32, 16, kernel_size=2, stride=2), nn.ReLU())
    self.up_4 = nn.Sequential(nn.ConvTranspose2d( 16,  8, kernel_size=2, stride=2), nn.ReLU())
    self.up_5 = nn.Sequential(nn.ConvTranspose2d(  8,  4, kernel_size=2, stride=2), nn.ReLU())
    self.up_6 = nn.Sequential(nn.ConvTranspose2d(  4,  2, kernel_size=2, stride=2), nn.ReLU())
    self.up_7 = nn.Sequential(nn.ConvTranspose2d(  2,  1, kernel_size=2, stride=2), nn.Sigmoid())

  def forward(self, x):              # ([1, 128])
    x = x.view(x.size(0), 128, 1, 1) # ([128,   1,   1])
    x = self.up_1(x)                 # ([ 64,   3,   3])
    x = self.up_2(x)                 # ([ 32,   7,   7])
    x = self.up_3(x)                 # ([ 16,  14,  14])
    x = self.up_4(x)                 # ([  8,  28,  28])
    x = self.up_5(x)                 # ([  4,  56,  56])
    x = self.up_6(x)                 # ([  2, 112, 112])
    x = self.up_7(x)                 # ([  1, 224, 224])

    return x

In [214]:
model = Decoder()
# model = ModelBase()
t = torch.randn((1,128))
print(t.shape)
get = model(t)
print(get.shape)

torch.Size([1, 128])
torch.Size([1, 1, 224, 224])
